In [1]:
# Import required libraries
import os
import sys
from pathlib import Path
import yaml
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import random
from tqdm import tqdm
import time

# Import YOLO
from ultralytics import YOLO

# Set up paths
notebook_dir = Path.cwd()
project_dir = notebook_dir.parent.parent
sys.path.append(str(project_dir))

print(f"📁 Notebook directory: {notebook_dir}")
print(f"📁 Project directory: {project_dir}")

# Dataset configuration
dataset_dir = Path("../../Datasets/ObjectDetectionSet/bdd100k/yolo_format")
config_file = dataset_dir / "data.yaml"

# Load dataset configuration
with open(config_file, 'r') as f:
    dataset_config = yaml.safe_load(f)

print("📋 Dataset Configuration:")
for key, value in dataset_config.items():
    print(f"  {key}: {value}")

# Class information
class_names = dataset_config['names']
num_classes = dataset_config['nc']

print(f"\n📋 Classes ({num_classes}): {class_names}")


📁 Notebook directory: /Users/romacarapetean/Desktop/Projects/Autonomous-Car-main/notebooks/object_detection
📁 Project directory: /Users/romacarapetean/Desktop/Projects/Autonomous-Car-main
📋 Dataset Configuration:
  path: /Users/romacarapetean/Desktop/Projects/Autonomous-Car-main/notebooks/object_detection/../../Datasets/ObjectDetectionSet/bdd100k/yolo_format
  train: images/train
  val: images/val
  test: images/test
  nc: 10
  names: ['pedestrian', 'rider', 'car', 'truck', 'bus', 'train', 'motorcycle', 'bicycle', 'traffic light', 'traffic sign']

📋 Classes (10): ['pedestrian', 'rider', 'car', 'truck', 'bus', 'train', 'motorcycle', 'bicycle', 'traffic light', 'traffic sign']


In [ ]:
# Load pre-trained YOLO model
print("🔄 Loading pre-trained YOLO model...")

# Load pre-trained YOLO model (will download if not available)
model = YOLO('yolov8n.pt')
print("✅ Loaded pre-trained YOLO model from ultralytics")

print(f"📊 Model info:")
print(f"  Model type: {type(model)}")

# Check model parameters
total_params = sum(p.numel() for p in model.model.parameters())
trainable_params = sum(p.numel() for p in model.model.parameters() if p.requires_grad)

print(f"  Total parameters: {total_params:,}")
print(f"  Trainable parameters: {trainable_params:,}")


🔄 Loading pre-trained YOLO model...
✅ Loaded model from: /Users/romacarapetean/Desktop/Projects/Autonomous-Car-main/notebooks/traffic_signs_detection/yolov8n.pt
📊 Model info:
  Model type: <class 'ultralytics.models.yolo.model.YOLO'>
  Total parameters: 3,157,200
  Trainable parameters: 0


In [ ]:
# Training configuration for testing
training_config = {
    'data': str(config_file),           # Dataset configuration file
    'epochs': 1,                        # Number of epochs (minimal for testing)
    'batch': 1,                         # Batch size (minimal for testing)
    'imgsz': 640,                       # Image size
    'device': 'cpu',                    # Use CPU for testing
    'workers': 0,                       # No multiprocessing for testing
    'project': '../../models/object_detection',  # Output directory
    'name': 'bdd100k_test',             # Experiment name
    'exist_ok': True,                   # Overwrite existing experiment
    'pretrained': True,                 # Use pre-trained weights
    'optimizer': 'auto',                # Auto-select optimizer
    'verbose': True,                    # Verbose output
    'seed': 42,                         # Random seed
    'deterministic': True,              # Deterministic training
    'single_cls': False,                # Multi-class detection
    'rect': False,                      # Rectangular training
    'cos_lr': False,                    # Cosine learning rate
    'close_mosaic': 0,                  # Disable mosaic augmentation
    'resume': False,                    # Don't resume from checkpoint
    'amp': False,                       # Disable mixed precision
    'fraction': 0.1,                    # Use only 10% of data for testing
    'cache': False,                     # Don't cache images
    'val': True,                        # Run validation
    'plots': True,                      # Generate plots
    'save': True,                       # Save results
    'save_period': -1,                  # Save only at end
}

print("📋 Training Configuration:")
for key, value in training_config.items():
    print(f"  {key}: {value}")

print(f"\n⚠️  Note: This is a minimal training run for testing purposes!")
print(f"   - Only 1 epoch")
print(f"   - Batch size of 1")
print(f"   - Using only 10% of data")
print(f"   - CPU training (slow but safe for testing)")


📋 Training Configuration:
  data: ../../Datasets/ObjectDetectionSet/bdd100k/yolo_format/data.yaml
  epochs: 1
  batch: 1
  imgsz: 640
  device: cpu
  workers: 0
  project: ../../models/object_detection
  name: bdd100k_test
  exist_ok: True
  pretrained: True
  optimizer: auto
  verbose: True
  seed: 42
  deterministic: True
  single_cls: False
  rect: False
  cos_lr: False
  close_mosaic: 0
  resume: False
  amp: False
  fraction: 0.1
  cache: False
  val: True
  plots: True
  save: True
  save_period: -1

⚠️  Note: This is a minimal training run for testing purposes!
   - Only 1 epoch
   - Batch size of 1
   - Using only 10% of data
   - CPU training (slow but safe for testing)


In [4]:
# Start training
print("🚀 Starting training...")
print("⏱️  This may take a while even with minimal settings...")

start_time = time.time()

try:
    # Train the model
    results = model.train(**training_config)
    
    training_time = time.time() - start_time
    print(f"✅ Training completed in {training_time:.2f} seconds")
    
    # Print training results
    print(f"\n📊 Training Results:")
    if hasattr(results, 'results_dict'):
        for key, value in results.results_dict.items():
            print(f"  {key}: {value}")
    
except Exception as e:
    print(f"❌ Training failed: {str(e)}")
    print(f"💡 This might be due to dataset issues or insufficient resources.")
    print(f"💡 Check the dataset configuration and try again.")


🚀 Starting training...
⏱️  This may take a while even with minimal settings...
New https://pypi.org/project/ultralytics/8.3.174 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.170 🚀 Python-3.13.1 torch-2.7.1 CPU (Apple M2)
engine/trainer: agnostic_nms=False, amp=False, augment=False, auto_augment=randaugment, batch=1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=0, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../../Datasets/ObjectDetectionSet/bdd100k/yolo_format/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=0.1, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/Users/romacarapetean/Des

train: Scanning /Users/romacarapetean/Desktop/Projects/Autonomous-Car-main/Datasets/ObjectDetectionSet/bdd100k/yolo_format/labels/train... 115 images, 0 backgrounds, 0 corrupt: 100%|██████████| 115/115 [00:00<00:00, 3431.08it/s]

train: New cache created: /Users/romacarapetean/Desktop/Projects/Autonomous-Car-main/Datasets/ObjectDetectionSet/bdd100k/yolo_format/labels/train.cache
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 179.1±58.3 MB/s, size: 51.7 KB)



val: Scanning /Users/romacarapetean/Desktop/Projects/Autonomous-Car-main/Datasets/ObjectDetectionSet/bdd100k/yolo_format/labels/val... 10000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 10000/10000 [00:02<00:00, 3724.69it/s]


val: New cache created: /Users/romacarapetean/Desktop/Projects/Autonomous-Car-main/Datasets/ObjectDetectionSet/bdd100k/yolo_format/labels/val.cache
Plotting labels to ../../models/object_detection/bdd100k_test/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to ../../models/object_detection/bdd100k_test
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1         0G      1.852      4.152      1.273         48        640: 100%|██████████| 115/115 [00:20<00:00,  5.52it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5000/5000 [08:16<00:00, 10.06it/s]


                   all      10000     185074     0.0136      0.138     0.0264     0.0174

1 epochs completed in 0.144 hours.
Optimizer stripped from ../../models/object_detection/bdd100k_test/weights/last.pt, 6.2MB
Optimizer stripped from ../../models/object_detection/bdd100k_test/weights/best.pt, 6.2MB

Validating ../../models/object_detection/bdd100k_test/weights/best.pt...
Ultralytics 8.3.170 🚀 Python-3.13.1 torch-2.7.1 CPU (Apple M2)
Model summary (fused): 72 layers, 3,007,598 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5000/5000 [07:41<00:00, 10.84it/s] 


                   all      10000     185074     0.0135      0.139     0.0264     0.0174
            pedestrian       3220      13262    0.00325     0.0112    0.00165   0.000943
                 rider        515        649    0.00017      0.133   0.000151   4.66e-05
                   car       9879     102506      0.112      0.322       0.22      0.146
                 truck       2689       4245    0.00289      0.332    0.00975    0.00569
                   bus       1242       1597    0.00269      0.292    0.00526    0.00362
                 train         14         15   5.92e-06     0.0667   3.94e-06   2.36e-06
               bicycle        578       1007   0.000418     0.0914   0.000321   0.000123
         traffic light       5653      26885    3.2e-05   0.000186   1.63e-05   4.27e-06
          traffic sign       8221      34908   0.000401    0.00358   0.000213   9.41e-05
Speed: 0.3ms preprocess, 36.9ms inference, 0.0ms loss, 4.3ms postprocess per image
Results saved to ../../mode

In [5]:
# Test on sample images
print("🔍 Testing model on sample images...")

test_images_dir = dataset_dir / "images/val"
if test_images_dir.exists():
    test_images = list(test_images_dir.glob('*.jpg'))
    
    if test_images:
        # Select random sample
        sample_image = random.choice(test_images)
        print(f"📸 Testing on: {sample_image.name}")
        
        # Make prediction
        print("🔍 Making prediction...")
        predictions = model(str(sample_image), conf=0.25, iou=0.45)
        
        if predictions:
            # Load and display image
            img = Image.open(sample_image)
            fig, ax = plt.subplots(1, 1, figsize=(12, 8))
            ax.imshow(img)
            
            # Draw predictions if any
            pred = predictions[0]
            if pred.boxes is not None and len(pred.boxes) > 0:
                for i in range(len(pred.boxes)):
                    # Get bounding box coordinates
                    x1, y1, x2, y2 = pred.boxes.xyxy[i].cpu().numpy()
                    confidence = pred.boxes.conf[i].cpu().numpy()
                    class_id = int(pred.boxes.cls[i].cpu().numpy())
                    
                    # Get class name
                    if class_id < len(class_names):
                        class_name = class_names[class_id]
                    else:
                        class_name = f"class_{class_id}"
                    
                    # Create rectangle
                    rect = patches.Rectangle(
                        (x1, y1), x2-x1, y2-y1, 
                        linewidth=2, 
                        edgecolor='red', 
                        facecolor='none'
                    )
                    ax.add_patch(rect)
                    
                    # Add label
                    label = f"{class_name} {confidence:.2f}"
                    ax.text(x1, y1-5, label, 
                            color='red', 
                            fontsize=10, fontweight='bold',
                            bbox=dict(boxstyle="round,pad=0.3", facecolor='white', alpha=0.8))
                
                print(f"✅ Detected {len(pred.boxes)} objects")
            else:
                print("⚠️  No objects detected")
            
            ax.set_title(f"Model Prediction - {sample_image.name}", fontsize=14, fontweight='bold')
            ax.axis('off')
            plt.tight_layout()
            plt.show()
        else:
            print("⚠️  No predictions returned")
    else:
        print(f"⚠️  No test images found in {test_images_dir}")
else:
    print(f"⚠️  Test images directory not found: {test_images_dir}")

print("\n🎉 Model building test completed!")
print("📁 Check the models directory for results and trained weights.")


🔍 Testing model on sample images...
📸 Testing on: becfc708-eaf36a96.jpg
🔍 Making prediction...

image 1/1 /Users/romacarapetean/Desktop/Projects/Autonomous-Car-main/notebooks/object_detection/../../Datasets/ObjectDetectionSet/bdd100k/yolo_format/images/val/becfc708-eaf36a96.jpg: 384x640 (no detections), 45.4ms
Speed: 1.4ms preprocess, 45.4ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)
⚠️  No objects detected


<Figure size 1200x800 with 1 Axes>


🎉 Model building test completed!
📁 Check the models directory for results and trained weights.


In [ ]:
# Evaluate model performance (if training was successful)
try:
    # Validate on a subset of data
    print("📊 Evaluating model performance...")
    
    val_results = model.val(
        data=str(config_file),
        device='cpu',
        verbose=True,
        save_json=True,
        save_hybrid=False,
        conf=0.001,
        iou=0.6,
        max_det=300,
        half=False,
        batch=1,
        project='../../models/object_detection',
        name='bdd100k_test_val'
    )
    
    if val_results:
        print(f"✅ Validation completed!")
        
        # Print key metrics
        metrics = val_results.results_dict
        if metrics:
            print(f"\n📊 Validation Metrics:")
            for key, value in metrics.items():
                if isinstance(value, (int, float)):
                    print(f"  {key}: {value:.4f}")
        
        # Get mAP scores
        if hasattr(val_results, 'box'):
            if hasattr(val_results.box, 'map'):
                print(f"\n📊 Key Performance Metrics:")
                print(f"  mAP@0.5: {val_results.box.map50:.4f}")
                print(f"  mAP@0.5:0.95: {val_results.box.map:.4f}")
                
                # Per-class mAP
                if hasattr(val_results.box, 'maps'):
                    print(f"\n📊 Per-class mAP@0.5:")
                    for i, class_map in enumerate(val_results.box.maps):
                        if i < len(class_names):
                            print(f"  {class_names[i]}: {class_map:.4f}")
    
except Exception as e:
    print(f"⚠️  Validation failed: {str(e)}")
    print(f"💡 This might be due to insufficient training or data issues.")

print(f"\n🏁 Model building and testing completed!")
print(f"📝 Summary:")
print(f"  - Loaded pre-trained YOLO model")
print(f"  - Configured for BDD100K dataset ({num_classes} classes)")
print(f"  - Ran minimal training (1 epoch, 1 batch)")
print(f"  - Tested on sample images")
print(f"  - Evaluated performance metrics")
print(f"\n💡 For full training, use the training script:")
print(f"   python scripts/object_detection/train_model.py --data Datasets/ObjectDetectionSet/bdd100k/yolo_format/data.yaml")


📊 Evaluating model performance...
WARNING ⚠️ 'save_hybrid' is deprecated and will be removed in in the future.
Ultralytics 8.3.170 🚀 Python-3.13.1 torch-2.7.1 CPU (Apple M2)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 176.2±64.6 MB/s, size: 57.4 KB)


val: Scanning /Users/romacarapetean/Desktop/Projects/Autonomous-Car-main/Datasets/ObjectDetectionSet/bdd100k/yolo_format/labels/val.cache... 10000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 10000/10000 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10000/10000 [06:30<00:00, 25.59it/s]


                   all      10000     185074     0.0142      0.141     0.0274     0.0178
            pedestrian       3220      13262    0.00305     0.0123    0.00156   0.000829
                 rider        515        649   0.000175      0.131   0.000156   4.64e-05
                   car       9879     102506      0.118      0.325      0.229       0.15
                 truck       2689       4245    0.00318      0.341     0.0102    0.00583
                   bus       1242       1597    0.00262      0.302    0.00545    0.00364
                 train         14         15   5.52e-06     0.0667   3.88e-06   2.33e-06
               bicycle        578       1007   0.000358     0.0914   0.000294   0.000106
         traffic light       5653      26885   4.31e-05   0.000298   2.18e-05   6.29e-06
          traffic sign       8221      34908   0.000388    0.00381   0.000207   8.57e-05
Speed: 0.2ms preprocess, 29.5ms inference, 0.0ms loss, 3.2ms postprocess per image
Saving ../../models/object_